In [1]:
import numpy as np
import pandas as pd
import csv
from scamp import *
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Flatten,Softmax,Input
import keras.backend as K 

Using TensorFlow backend.


In [2]:
df = pd.read_csv('Species.csv', sep=",",index_col = 0)
print(df)

      Cantus  Counter
0         58       67
1         66       69
2         64       71
3         68       76
4         66       74
...      ...      ...
9995      64       72
9996      56       60
9997      57       61
9998      61       65
9999      56       64

[10000 rows x 2 columns]


In [3]:
n = df.shape[0]
n

10000

In [4]:
X = tf.Variable(np.zeros((2,n*2)),dtype='int32')  # any data tensor
for i in range(0,2*n,2):
    X[0,i].assign(df['Cantus'][i/2] - 43)
    X[1,i].assign(0)
    X[0,i+1].assign(df['Counter'][i/2] - 55)
    X[1,i+1].assign(1)
X

<tf.Variable 'Variable:0' shape=(2, 20000) dtype=int32, numpy=
array([[15, 12, 23, ..., 10, 13,  9],
       [ 0,  1,  0, ...,  1,  0,  1]])>

In [5]:
depth = 30
X = tf.concat([tf.one_hot(X[0,:], depth),tf.dtypes.cast(tf.reshape(X[1,:],[20000,1]),tf.float32)],1)
X

<tf.Tensor: shape=(20000, 31), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)>

In [6]:
train_dataset = tf.data.Dataset.from_tensor_slices(X[:16000,:])
valid_dataset = tf.data.Dataset.from_tensor_slices(X[16000:19000,:])
test_dataset = tf.data.Dataset.from_tensor_slices(X[19000:,:])

In [43]:
def data_process(dataset,batch_size):
    n_steps = 20
    window_length = n_steps + 2 # target = input shifted 1 character ahead
    dataset = dataset.window(window_length, shift=2, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    
    shuffle_dataset = dataset.shuffle(20000).batch(batch_size,drop_remainder=True)
    #map_dataset = shuffle_dataset.map(lambda windows: (windows[:,:-2,:], windows[:,-2:-1,:],windows[:,1:-1,:-1], windows[:,-2,:],windows[:,-1,:]))
    map_dataset = shuffle_dataset.map(lambda windows: (windows[:,:-2,:], windows[:,-1:,:],tf.concat((windows[:,1:-2,:-1],windows[:,-1:,:-1]),1),windows[:,-1,:],windows[:,-2,:]))
    final_dataset = map_dataset.map(
        lambda X_batch, Y_batch, Z_batch, U_batch, V_batch: ((X_batch, Y_batch), (Z_batch, U_batch[:,:-1], V_batch[:,:-1])))
    return final_dataset

In [10]:
batch_size = 20
train = data_process(train_dataset,batch_size)
valid = data_process(valid_dataset,batch_size)
test = data_process(test_dataset,batch_size)

NameError: name 'data_process' is not defined

In [11]:
rnn_units = 12
n_notes = 30

input_A = keras.layers.Input(shape=(batch_size,31), name="input_A") # (15, 20, 31)
input_B = keras.layers.Input(shape=(1,31), name="input_B") # (15, 1, 31)

hidden1 = keras.layers.LSTM(rnn_units, return_sequences=True)(input_A) # (15,20,12)
hidden2 = keras.layers.LSTM(rnn_units, return_sequences=True)(hidden1) #(15,20,12)

output_RNN = keras.layers.Dense(n_notes, activation='softmax', name = 'output_RNN')(hidden2) #(15,20,30)

e = keras.layers.Dense(1, activation='tanh')(hidden2) #(15,20,1)
e = keras.layers.Reshape([-1])(e) #(15,20)

alpha = keras.layers.Activation('softmax')(e) #(15,20)
c = keras.layers.Permute([2, 1])(keras.layers.RepeatVector(rnn_units)(alpha)) #(15,20,12)
c = keras.layers.Multiply()([hidden2, c]) #(15,20,12)
c = keras.layers.Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c) #(15,12)

output_A = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_A')(c) #(15,30) (0, cantus)
reshape = tf.reshape(output_A,[-1,1,30]) # (15, 1, 30)

#zero = tf.reshape(tf.zeros(batch_size),[batch_size,1,1])
#input_output_A = tf.concat((reshape,zero),2) # (15, 1, 31)  (0, cantus)

one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
input_output_A = tf.concat((reshape,one),2) # (15, 1, 31)  (0, cantus)

aux1 = keras.layers.SimpleRNN(rnn_units)(input_B, initial_state=c) #(15,12)
aux2 = keras.layers.SimpleRNN(rnn_units)(input_output_A, initial_state=c) #(15,12)
output_B1 = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_B1')(aux1) #(15,30) (1, counter)
output_B2 = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_B2')(aux2) #(15,30) (1, counter)

model = keras.models.Model([input_A, input_B], [output_RNN, output_A, output_B1])
aux_model2 = keras.models.Model(input_A, [output_A, output_B2])
sub_model = keras.models.Model(input_A, output_A)

model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy','categorical_crossentropy'],
          loss_weights=[0.1, 0.3, 0.3, 0.3],optimizer="sgd")


In [12]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_A (InputLayer)            [(None, 20, 31)]     0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 20, 12)       2112        input_A[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 20, 12)       1200        lstm[0][0]                       
__________________________________________________________________________________________________
dense (Dense)                   (None, 20, 1)        13          lstm_1[0][0]                     
_______________________________________________________________________________________

In [47]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("model_1_0_partial.h5",
                                                save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)
history = model.fit(train, epochs=1000,
                    validation_data = valid,
                    callbacks=[checkpoint_cb, early_stopping_cb])
# model = keras.models.load_model("my_keras_model.h5") # roll back to best model

Epoch 1/1000
399/399 [==============================] - 3s 9ms/step - loss: 2.3834 - output_RNN_loss: 3.4017 - output_A_loss: 3.3932 - output_B1_loss: 3.4177 - val_loss: 2.3742 - val_output_RNN_loss: 3.4015 - val_output_A_loss: 3.3846 - val_output_B1_loss: 3.3955
Epoch 2/1000
399/399 [==============================] - 3s 7ms/step - loss: 2.3707 - output_RNN_loss: 3.4020 - output_A_loss: 3.3798 - output_B1_loss: 3.3887 - val_loss: 2.3625 - val_output_RNN_loss: 3.4014 - val_output_A_loss: 3.3720 - val_output_B1_loss: 3.3691
Epoch 3/1000
399/399 [==============================] - 3s 7ms/step - loss: 2.3608 - output_RNN_loss: 3.4018 - output_A_loss: 3.3690 - output_B1_loss: 3.3664 - val_loss: 2.3527 - val_output_RNN_loss: 3.4007 - val_output_A_loss: 3.3615 - val_output_B1_loss: 3.3473
Epoch 4/1000
399/399 [==============================] - 3s 7ms/step - loss: 2.3522 - output_RNN_loss: 3.4013 - output_A_loss: 3.3598 - output_B1_loss: 3.3470 - val_loss: 2.3441 - val_output_RNN_loss: 3.3996 -

399/399 [==============================] - 10s 25ms/step - loss: 2.1006 - output_RNN_loss: 3.3325 - output_A_loss: 3.2446 - output_B1_loss: 2.6465 - val_loss: 2.0887 - val_output_RNN_loss: 3.3225 - val_output_A_loss: 3.2320 - val_output_B1_loss: 2.6229
Epoch 62/1000
399/399 [==============================] - 10s 26ms/step - loss: 2.0951 - output_RNN_loss: 3.3314 - output_A_loss: 3.2430 - output_B1_loss: 2.6304 - val_loss: 2.0835 - val_output_RNN_loss: 3.3215 - val_output_A_loss: 3.2292 - val_output_B1_loss: 2.6085
Epoch 63/1000
399/399 [==============================] - 10s 25ms/step - loss: 2.0899 - output_RNN_loss: 3.3304 - output_A_loss: 3.2417 - output_B1_loss: 2.6145 - val_loss: 2.0776 - val_output_RNN_loss: 3.3208 - val_output_A_loss: 3.2278 - val_output_B1_loss: 2.5907
Epoch 64/1000
399/399 [==============================] - 10s 26ms/step - loss: 2.0845 - output_RNN_loss: 3.3294 - output_A_loss: 3.2397 - output_B1_loss: 2.5989 - val_loss: 2.0738 - val_output_RNN_loss: 3.3194 - v

399/399 [==============================] - 10s 25ms/step - loss: 1.8468 - output_RNN_loss: 3.2124 - output_A_loss: 3.0503 - output_B1_loss: 2.0347 - val_loss: 1.8397 - val_output_RNN_loss: 3.2050 - val_output_A_loss: 3.0437 - val_output_B1_loss: 2.0204
Epoch 122/1000
399/399 [==============================] - 10s 25ms/step - loss: 1.8439 - output_RNN_loss: 3.2106 - output_A_loss: 3.0466 - output_B1_loss: 2.0296 - val_loss: 1.8379 - val_output_RNN_loss: 3.2019 - val_output_A_loss: 3.0398 - val_output_B1_loss: 2.0192
Epoch 123/1000
399/399 [==============================] - 10s 25ms/step - loss: 1.8407 - output_RNN_loss: 3.2086 - output_A_loss: 3.0420 - output_B1_loss: 2.0242 - val_loss: 1.8341 - val_output_RNN_loss: 3.2000 - val_output_A_loss: 3.0351 - val_output_B1_loss: 2.0119
Epoch 124/1000
399/399 [==============================] - 10s 25ms/step - loss: 1.8381 - output_RNN_loss: 3.2066 - output_A_loss: 3.0383 - output_B1_loss: 2.0197 - val_loss: 1.8326 - val_output_RNN_loss: 3.1976 

Epoch 182/1000
399/399 [==============================] - 3s 6ms/step - loss: 1.7209 - output_RNN_loss: 3.0817 - output_A_loss: 2.8682 - output_B1_loss: 1.8410 - val_loss: 1.7215 - val_output_RNN_loss: 3.0784 - val_output_A_loss: 2.8734 - val_output_B1_loss: 1.8390
Epoch 183/1000
399/399 [==============================] - 2s 6ms/step - loss: 1.7199 - output_RNN_loss: 3.0791 - output_A_loss: 2.8669 - output_B1_loss: 1.8398 - val_loss: 1.7239 - val_output_RNN_loss: 3.0792 - val_output_A_loss: 2.8772 - val_output_B1_loss: 1.8426
Epoch 184/1000
399/399 [==============================] - 2s 6ms/step - loss: 1.7184 - output_RNN_loss: 3.0768 - output_A_loss: 2.8649 - output_B1_loss: 1.8375 - val_loss: 1.7195 - val_output_RNN_loss: 3.0796 - val_output_A_loss: 2.8698 - val_output_B1_loss: 1.8355
Epoch 185/1000
399/399 [==============================] - 3s 6ms/step - loss: 1.7172 - output_RNN_loss: 3.0752 - output_A_loss: 2.8631 - output_B1_loss: 1.8357 - val_loss: 1.7183 - val_output_RNN_loss: 

Epoch 213/1000
399/399 [==============================] - 2s 6ms/step - loss: 1.6829 - output_RNN_loss: 3.0181 - output_A_loss: 2.8125 - output_B1_loss: 1.7912 - val_loss: 1.6899 - val_output_RNN_loss: 3.0194 - val_output_A_loss: 2.8293 - val_output_B1_loss: 1.7971
Epoch 214/1000
399/399 [==============================] - 2s 6ms/step - loss: 1.6818 - output_RNN_loss: 3.0154 - output_A_loss: 2.8112 - output_B1_loss: 1.7897 - val_loss: 1.6883 - val_output_RNN_loss: 3.0185 - val_output_A_loss: 2.8280 - val_output_B1_loss: 1.7934
Epoch 215/1000
399/399 [==============================] - 2s 6ms/step - loss: 1.6809 - output_RNN_loss: 3.0140 - output_A_loss: 2.8097 - output_B1_loss: 1.7886 - val_loss: 1.6856 - val_output_RNN_loss: 3.0142 - val_output_A_loss: 2.8223 - val_output_B1_loss: 1.7915
Epoch 216/1000
399/399 [==============================] - 3s 6ms/step - loss: 1.6800 - output_RNN_loss: 3.0117 - output_A_loss: 2.8085 - output_B1_loss: 1.7877 - val_loss: 1.6871 - val_output_RNN_loss: 

...

Epoch 337/1000
399/399 [==============================] - 2s 6ms/step - loss: 1.6070 - output_RNN_loss: 2.8709 - output_A_loss: 2.7073 - output_B1_loss: 1.6922 - val_loss: 1.6216 - val_output_RNN_loss: 2.8760 - val_output_A_loss: 2.7374 - val_output_B1_loss: 1.7091
Epoch 338/1000
399/399 [==============================] - 2s 6ms/step - loss: 1.6069 - output_RNN_loss: 2.8691 - output_A_loss: 2.7066 - output_B1_loss: 1.6933 - val_loss: 1.6239 - val_output_RNN_loss: 2.8775 - val_output_A_loss: 2.7383 - val_output_B1_loss: 1.7155
Epoch 339/1000
399/399 [==============================] - 2s 6ms/step - loss: 1.6065 - output_RNN_loss: 2.8682 - output_A_loss: 2.7063 - output_B1_loss: 1.6927 - val_loss: 1.6206 - val_output_RNN_loss: 2.8783 - val_output_A_loss: 2.7343 - val_output_B1_loss: 1.7082
Epoch 340/1000
399/399 [==============================] - 2s 6ms/step - loss: 1.6062 - output_RNN_loss: 2.8681 - output_A_loss: 2.7062 - output_B1_loss: 1.6917 - val_loss: 1.6177 - val_output_RNN_loss: 

Epoch 368/1000
399/399 [==============================] - 2s 6ms/step - loss: 1.5959 - output_RNN_loss: 2.8452 - output_A_loss: 2.6925 - output_B1_loss: 1.6787 - val_loss: 1.6074 - val_output_RNN_loss: 2.8532 - val_output_A_loss: 2.7149 - val_output_B1_loss: 1.6921
Epoch 369/1000
399/399 [==============================] - 2s 6ms/step - loss: 1.5956 - output_RNN_loss: 2.8445 - output_A_loss: 2.6919 - output_B1_loss: 1.6788 - val_loss: 1.6158 - val_output_RNN_loss: 2.8510 - val_output_A_loss: 2.7384 - val_output_B1_loss: 1.6972
Epoch 370/1000
399/399 [==============================] - 2s 6ms/step - loss: 1.5955 - output_RNN_loss: 2.8440 - output_A_loss: 2.6904 - output_B1_loss: 1.6798 - val_loss: 1.6094 - val_output_RNN_loss: 2.8518 - val_output_A_loss: 2.7186 - val_output_B1_loss: 1.6954
Epoch 371/1000
399/399 [==============================] - 2s 6ms/step - loss: 1.5952 - output_RNN_loss: 2.8434 - output_A_loss: 2.6900 - output_B1_loss: 1.6795 - val_loss: 1.6076 - val_output_RNN_loss: 

Epoch 399/1000
399/399 [==============================] - 2s 6ms/step - loss: 1.5864 - output_RNN_loss: 2.8223 - output_A_loss: 2.6780 - output_B1_loss: 1.6692 - val_loss: 1.6018 - val_output_RNN_loss: 2.8308 - val_output_A_loss: 2.7104 - val_output_B1_loss: 1.6855
Epoch 400/1000
399/399 [==============================] - 3s 6ms/step - loss: 1.5862 - output_RNN_loss: 2.8221 - output_A_loss: 2.6778 - output_B1_loss: 1.6689 - val_loss: 1.6006 - val_output_RNN_loss: 2.8296 - val_output_A_loss: 2.7060 - val_output_B1_loss: 1.6861
Epoch 401/1000
399/399 [==============================] - 3s 6ms/step - loss: 1.5856 - output_RNN_loss: 2.8209 - output_A_loss: 2.6776 - output_B1_loss: 1.6673 - val_loss: 1.5993 - val_output_RNN_loss: 2.8284 - val_output_A_loss: 2.7023 - val_output_B1_loss: 1.6858
Epoch 402/1000
399/399 [==============================] - 2s 6ms/step - loss: 1.5859 - output_RNN_loss: 2.8201 - output_A_loss: 2.6794 - output_B1_loss: 1.6670 - val_loss: 1.5996 - val_output_RNN_loss: 

Epoch 430/1000
399/399 [==============================] - 10s 25ms/step - loss: 1.5779 - output_RNN_loss: 2.8006 - output_A_loss: 2.6667 - output_B1_loss: 1.6593 - val_loss: 1.5939 - val_output_RNN_loss: 2.8098 - val_output_A_loss: 2.6947 - val_output_B1_loss: 1.6818
Epoch 431/1000
399/399 [==============================] - 10s 25ms/step - loss: 1.5777 - output_RNN_loss: 2.8001 - output_A_loss: 2.6664 - output_B1_loss: 1.6592 - val_loss: 1.5925 - val_output_RNN_loss: 2.8093 - val_output_A_loss: 2.6935 - val_output_B1_loss: 1.6785
Epoch 432/1000
399/399 [==============================] - 10s 25ms/step - loss: 1.5769 - output_RNN_loss: 2.7990 - output_A_loss: 2.6652 - output_B1_loss: 1.6582 - val_loss: 1.5946 - val_output_RNN_loss: 2.8080 - val_output_A_loss: 2.6994 - val_output_B1_loss: 1.6799
Epoch 433/1000
399/399 [==============================] - 10s 25ms/step - loss: 1.5774 - output_RNN_loss: 2.7981 - output_A_loss: 2.6664 - output_B1_loss: 1.6588 - val_loss: 1.5916 - val_output_RN

399/399 [==============================] - 10s 24ms/step - loss: 1.5633 - output_RNN_loss: 2.7628 - output_A_loss: 2.6471 - output_B1_loss: 1.6431 - val_loss: 1.5807 - val_output_RNN_loss: 2.7754 - val_output_A_loss: 2.6764 - val_output_B1_loss: 1.6675
Epoch 491/1000
399/399 [==============================] - 10s 25ms/step - loss: 1.5635 - output_RNN_loss: 2.7619 - output_A_loss: 2.6466 - output_B1_loss: 1.6444 - val_loss: 1.5807 - val_output_RNN_loss: 2.7743 - val_output_A_loss: 2.6772 - val_output_B1_loss: 1.6669
Epoch 492/1000
399/399 [==============================] - 10s 25ms/step - loss: 1.5629 - output_RNN_loss: 2.7616 - output_A_loss: 2.6456 - output_B1_loss: 1.6437 - val_loss: 1.5852 - val_output_RNN_loss: 2.7765 - val_output_A_loss: 2.6838 - val_output_B1_loss: 1.6748
Epoch 493/1000
399/399 [==============================] - 10s 25ms/step - loss: 1.5628 - output_RNN_loss: 2.7615 - output_A_loss: 2.6459 - output_B1_loss: 1.6431 - val_loss: 1.5816 - val_output_RNN_loss: 2.7743 

In [36]:
total_loss, output_RNN_loss, output_A_loss, output_B1_loss, output_B2_loss = model.evaluate(test)

24/24 [==============================] - 0s 3ms/step - loss: 2.3412 - output_RNN_loss: 2.7116 - output_A_loss: 2.6246 - output_B1_loss: 1.6881 - output_B2_loss: 2.5874


In [51]:
model.save("model_1_0.h5")
sub_model.save("sub_model_1_0.h5")
att_model.save("att_model_1_0.h5")

In [48]:
model.save_weights('model_1_0_partial_weight.h5')

In [13]:
model.load_weights('model_1_0_partial_weight.h5')

In [14]:
X_start_A = tf.reshape(X[19000:19020,:],[1,20,31])
[output_A, output_B2] = aux_model2.predict(X_start_A).reshape([30,])
output_A

InvalidArgumentError:  ConcatOp : Dimensions of inputs should match: shape[0] = [1,1,30] vs. shape[1] = [20,1,1]
	 [[node functional_3/tf_op_layer_concat/concat (defined at <ipython-input-14-422bd8fd0ab4>:2) ]] [Op:__inference_predict_function_242697]

Function call stack:
predict_function


In [48]:
pred_A_class_random = np.random.choice(30, 1, p=output_A)[0]
pred_A_class_random

6

In [49]:
one_hot = tf.reshape(tf.one_hot(pred_A_class_random, 30),[1,1,30])
out_0 = tf.concat((one_hot,[[[0]]]),2)
out_0

<tf.Tensor: shape=(1, 1, 31), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]],
      dtype=float32)>

In [50]:
y_pred_A, y_pred_B = model.predict((X_start_A,out_0))
print('y_pred_A:',y_pred_A, 'y_pred_B:',y_pred_B)

y_pred_A: [[0.03155782 0.0325329  0.03173704 0.03192566 0.03506639 0.03417719
  0.0338481  0.03291002 0.03165215 0.03471051 0.03455314 0.03086988
  0.03325387 0.03299863 0.03273064 0.0354063  0.03468685 0.03317405
  0.03461605 0.03439218 0.03557383 0.03242486 0.03168539 0.03368177
  0.03217451 0.03288629 0.03588217 0.03389754 0.03139951 0.03359476]] y_pred_B: [[0.03185337 0.03439081 0.02634412 0.02846782 0.03581786 0.0269027
  0.03651585 0.03407611 0.0364597  0.03747211 0.02995922 0.0256372
  0.0303991  0.0301232  0.02858168 0.02684872 0.04141647 0.03798914
  0.0398167  0.03374988 0.04703412 0.02937686 0.03111148 0.03577967
  0.03487676 0.04168951 0.02990909 0.03666779 0.03080825 0.02992463]]


In [36]:
def generate_sequence(initial,seq_length):
    """
    Arguments:
    initial -- initial sequence of shape (1,20,31) (1 sample, sequence of length 20, vector fearure: one-hot(30);cantus(0),counter(1))
    seq_length -- # pairs of notes generated
    
    """
    ### START CODE HERE ###
    sequence = initial
    for i in range(seq_length):
        output_A = sub_model.predict(initial).reshape([30,])
        pred_A_class_random = np.random.choice(30, 1, p=output_A)[0]
        pred_A_class_max = np.where(output_A == np.max(output_A))[0][0]

        one_hot = tf.reshape(tf.one_hot(pred_A_class_random, 30),[1,1,30])
        out_0 = tf.concat((one_hot,[[[1]]]),2)
        sequence = tf.concat((sequence,out_0),1)

        y_pred_RNN, y_pred_A, y_pred_B = model.predict((initial,out_0))
        output_B = y_pred_B.reshape([30,])
        pred_B_class_random = np.random.choice(30, 1, p=output_B)[0]
        pred_B_class_max = np.where(output_B == np.max(output_B))[0][0]

        one_hot = tf.reshape(tf.one_hot(pred_B_class_random, 30),[1,1,30])
        out_1 = tf.concat((one_hot,[[[0]]]),2)
        sequence = tf.concat((sequence,out_1),1)

        initial = sequence[:,-20:,:]
        
    return sequence

In [44]:
X_start_A = tf.reshape(X[19000:19020,:],[1,20,31])
seq_length = 10
sequence = generate_sequence(X_start_A,seq_length)

In [45]:
new_sequence = sequence[:,20:,:]
sequence

<tf.Tensor: shape=(1, 40, 31), dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)>

In [46]:
sequence = tf.reshape(sequence,[20+2*seq_length,31])
sequence = np.array(sequence)
sequence.shape

(40, 31)

In [47]:
sequence = sequence[-20:,:]
row,col = sequence.shape
cantus_lst = [];
counter_lst = [];
for i in range(row):
    clas = np.where(sequence[i,:-1] == 1)[0][0]
    if sequence[i,-1] == 0:
        cantus_lst.append(clas+43)
    elif sequence[i,-1] == 1:
        counter_lst.append(clas+55)

In [48]:
cantus_lst

[67, 68, 69, 71, 66, 63, 67, 62, 63, 62]

In [49]:
counter_lst

[77, 77, 73, 79, 78, 67, 73, 66, 67, 66]

In [50]:
np.array(counter_lst) - np.array(cantus_lst) # real harmonic

array([10,  9,  4,  8, 12,  4,  6,  4,  4,  4], dtype=int64)

In [71]:
s = Session()
s.tempo = 180
piano1 = s.new_part("piano")
piano2 = s.new_part("piano")

def cantus():
    for i in cantus_lst:
        piano2.play_note(i,1,4)
        
def counter():
    for i in counter_lst:
        piano1.play_note(i,1,4)
        
        
s.fast_forward_to_beat(100)

s.start_transcribing()
s.fork(counter)
s.fork(cantus)
s.wait_for_children_to_finish()
performance = s.stop_transcribing()
performance.to_score(title = "First Species Counterpoint", composer = "My programme",time_signature = "4/4").show_xml()

Using preset Piano Merlin for piano
Using preset Piano Merlin for piano


In [72]:
np.array(counter_lst) - np.array(cantus_lst) # harmonic

array([4, 3, 9, 8, 3, 8, 8, 9, 9, 3, 8, 4, 4, 7, 7, 6, 9, 7, 8, 3],
      dtype=int64)

In [73]:
np.diff(cantus_lst)

array([ -1,  -4,  -2,   3,   3,  -1,  -2,   3,  10, -15,  13,  -4,   0,
        -2,  -5,  -1,  -1,  -1,  12], dtype=int64)

In [74]:
np.diff(counter_lst)

array([ -2,   2,  -3,  -2,   8,  -1,  -1,   3,   4, -10,   9,  -4,   3,
        -2,  -6,   2,  -3,   0,   7], dtype=int64)

In [32]:
model1 = keras.models.load_model("my_keras_model.h5")

In [35]:
sub = keras.models.load_model("sub_model.h5")

In [37]:
X_start_A = tf.reshape(X[9000:9020,:],[1,20,31])
sub.predict(X_start_A).reshape([30,])

array([0.0002585 , 0.00053626, 0.0003319 , 0.0008722 , 0.00056339,
       0.00058614, 0.00078768, 0.00098308, 0.00130475, 0.00173629,
       0.00228998, 0.00487447, 0.00683993, 0.00934752, 0.01578839,
       0.01636755, 0.02636185, 0.0402819 , 0.05282699, 0.06961218,
       0.06363309, 0.07971542, 0.08033299, 0.09574662, 0.08555523,
       0.08963284, 0.07672965, 0.07218166, 0.04997258, 0.053949  ],
      dtype=float32)

In [39]:
att_model.predict((X_start_A,out_0))

array([[0.02218576, 0.02161889, 0.02157068, 0.02155778, 0.02155401,
        0.02155203, 0.02155126, 0.02155054, 0.02155021, 0.02154984,
        0.02154955, 0.02154944, 0.02154973, 0.0215589 , 0.02210564,
        0.04799915, 0.15077986, 0.15857086, 0.1590038 , 0.1590921 ]],
      dtype=float32)

In [74]:
out_A = tf.reshape(X[9020,:-1],[1,30])
history = sub_model.fit(X_start_A,out_A)

1/1 [==============================] - 0s 2ms/step - loss: 2.4345


In [75]:
model.load_weights('model_0_1_weight.h5')

ValueError: You are trying to load a weight file containing 6 layers into a model with 4 layers.